## Parâmetros do modelo para iterações

In [41]:
preco_qav                 = 4.49
preco_dolar               = 5.99     # R$
valor_hefa_soja           = 1.13 # US$/L
quanto_tarifa_combustivel = 0.30

dict_preco_saf_dolar_por_litro = {
    'hefa_soja': 1.13,
    'hefa_macauba': 0.57,
    'hefa_palma': 1.13,
    'hefa_UCO': 1.32,
    'atj_cana': 1.86,
    'atj_milho': 1.27    
} # US$/L
dict_preco_saf_real_litro = {k: v * preco_dolar for k, v in dict_preco_saf_dolar_por_litro.items()}# R$/L

dict_elasticidade_preco = {
    'historico': -1.47,
    'boom_economico': -1.29,
    'recessao': -1.59,
}

fator_emissao_qav = 89 #emissao querosene fóssil
dict_fator_emissao_safs = {
    'hefa_soja_sem_luc': 20.1,
    'hefa_soja_com_luc': 47.4,
    'hefa_macauba_sem_luc': 53.7,
    'hefa_macauba_com_luc': 71,
    'hefa_palma_sem_luc': 48.3,
    'hefa_palma_com_luc': 94.5,
    'hefa_UCO': 21.2,
    'atj_cana_sem_luc': 22.4,
    'atj_cana_com_luc': 45.6,
    'atj_milho_sem_luc': 39.5,
    'atj_milho_com_luc': 81
}

dict_rota_materia_prima = { # chave: valor_materia_prima, valor: fator_emissao_materia_prima
    'hefa_soja':    ['hefa_soja_sem_luc', 'hefa_soja_com_luc'],
    'hefa_macauba': ['hefa_macauba_sem_luc', 'hefa_macauba_com_luc'],
    'hefa_palma':   ['hefa_palma_sem_luc', 'hefa_palma_com_luc'],
    'hefa_UCO':     ['hefa_UCO'],
    'atj_cana':     ['atj_cana_sem_luc', 'atj_cana_com_luc'],
    'atj_milho':    ['atj_milho_sem_luc', 'atj_milho_com_luc']
}

dict_mandato_emissao_por_ano = {
    2030: 0.03,
    2040: 0.13,
    2050: 0.23,
}


In [42]:
nome_excel = 'Cenários_dissertação_PC.xlsm'
nome_aba_preco_qav = 'Preço do QAV'

## 1. estimar a entrada da SAF, ver o diferencial de valor em relação ao QAV
## 2. transmitir esse aumento para tarifa

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [44]:
potencial_energetico = pd.read_excel('Cenários_dissertação_PC.xlsm', sheet_name='Potencial energético')
potencial_energetico.columns = range(potencial_energetico.shape[1])
# potencial_energetico.head(5)

# Transforma a primeira linha em cabeçalho e filtra colunas Ano e Potencial Combustível Necessário
for i in range(1, potencial_energetico.iloc[0, 1:].shape[0], 2):
    if i > potencial_energetico.iloc[0, 1:].shape[0]:
        break
    nome_coluna = potencial_energetico.iloc[0, i]
    potencial_energetico = potencial_energetico.drop(potencial_energetico.columns[[i, i+1]], axis=1)
    potencial_energetico.iloc[0, i] = nome_coluna
    potencial_energetico.head()

potencial_energetico.columns = potencial_energetico.iloc[0]
potencial_energetico = potencial_energetico.dropna( axis=1, how='all').drop([0, 1]).reset_index(drop=True)
todas_demandas_intensidade_energetica = potencial_energetico.columns[1:].to_list()
print(f'{todas_demandas_intensidade_energetica = }')

potencial_energetico.tail(2)

todas_demandas_intensidade_energetica = ['BAU-Base', 'BAU-Moderada', 'BAU-Acelerada', 'Retração-Base', 'Retração-Moderada', 'Retração-Acelerada', 'Conexões-Base', 'Conexões-Moderada', 'Conexões-Acelerada']


,Ano,BAU-Base,BAU-Moderada,BAU-Acelerada,Retração-Base,Retração-Moderada,Retração-Acelerada,Conexões-Base,Conexões-Moderada,Conexões-Acelerada
49,2049,355900329443.074951,338001143972.582275,307172133146.176636,313457685426.678162,297693054757.796326,270540535981.651764,441956760359.358459,419729565357.912048,381446122992.902588
50,2050,360041221842.48999,340559265044.504333,307858101048.461426,315498123545.477539,298426409419.789185,269770924290.317017,450988714163.252991,426585556656.051392,385623980571.108276


1. estimar a entrada da SAF, ver o diferencial de valor em relação ao QAV
2. transmitir esse aumento para tarifa
3. assumir a elasticidade preço da demanda do paper que compartilhou (por exemplo, assumir o valor do Brasil para o período sem recessão, para ser conservador)
4. estimar a queda da demanda
5. estimar a nova demanda de saf
fazer até que não gera alteração significativa na demanda por SAF.
Isso mostraria quanto, sem medidas de amortecimento do aumento de custo, a demanda poderia cair e consequentemente o mercado de QAV e SAF, claro.


In [45]:
def iteracao(df_iteracao: pd.DataFrame,
             numero_iteracao: np.int64, 
             nome_demanda_intensidade_energetica: str, 
             elasticidade_preco: np.float64, 
             nome_preco_saf: str,
             nome_fator_emissao_saf: str,
             demanda_energetica_anterior: np.float64, 
             quanto_combustivel_eh_saf: np.float64, 
             mandato_emissao_por_ano: dict[int: np.float64],
             ano_referencia: np.int32 = 2019):
    preco_saf = dict_preco_saf_real_litro[nome_preco_saf]
    aumento_preco_litro                  = preco_saf / preco_qav - 1
    aumento_custo_saf                    = aumento_preco_litro * quanto_combustivel_eh_saf
    variacao_preco                       = aumento_custo_saf * quanto_tarifa_combustivel
    variacao_demanda                     = variacao_preco * elasticidade_preco
    emissao_referencia                   = potencial_energetico[potencial_energetico['Ano'] == ano_referencia][nome_demanda_intensidade_energetica].values[0] * fator_emissao_qav
    aumento_custo_saf                    = aumento_preco_litro * quanto_combustivel_eh_saf
    fator_emissao_dif = fator_emissao_qav - dict_fator_emissao_safs[nome_fator_emissao_saf]
#     df                                   = pd.DataFrame([nome_demanda_intensidade_energetica for _ in mandato_emissao_por_ano], columns=['nome_demanda_intensidade_energetica'])
#     df['elasticidade_preco']             = elasticidade_preco
    df                                   = pd.DataFrame([elasticidade_preco for _ in mandato_emissao_por_ano], columns=['elasticidade_preco'])
    df['nome_preco_saf']                 = nome_preco_saf
    df['fator_emissao_saf']              = nome_fator_emissao_saf
    df['nome_demanda_intensidade_energetica']             = nome_demanda_intensidade_energetica
    df['Ano']                            = mandato_emissao_por_ano.keys()
    df['mandato_emissao']                = mandato_emissao_por_ano.values()
    df['numero_iteracao']                = numero_iteracao
    df['demanda_intensidade_energetica'] = (1 + variacao_demanda) * demanda_energetica_anterior
    df['emissao_tudo_qav']               = df['demanda_intensidade_energetica'] * fator_emissao_qav
    df['reducao_emissao_referencia']     = df['mandato_emissao'] * emissao_referencia
    df['reducao_via_saf_ano_referencia'] = df['emissao_tudo_qav'] - emissao_referencia
    df['reducao_via_saf_ano_destino']    = df['reducao_via_saf_ano_referencia'] + df['reducao_emissao_referencia']
    df['demanda_energetica_saf']         = df['reducao_via_saf_ano_destino'] / fator_emissao_dif
    df['variacao_demanda']               = variacao_demanda
    df['percentual_energetico_saf']      = df['demanda_energetica_saf'] / df['demanda_intensidade_energetica']

    df_iteracao = pd.concat([df_iteracao, df])
    return df_iteracao.reset_index(drop=True)


In [46]:
def iteracao_dissertacao(lista_nome_demanda_intensidade_energetica: list[str],
                         dict_rota_materia_prima: dict[str: list[str]],
                        #  dict_preco_saf_real_litro: dict[str: np.float64],
                        #  dict_fator_emissao_safs: dict[str: np.float64],
                         lista_elasticidades: list[np.float64], 
                         dict_mandato_emissao_por_ano: dict[int: np.float64], 
                         ano_referencia: np.int64 = 2019):
    assert len(lista_nome_demanda_intensidade_energetica) > 0, 'A lista de nomes de demanda intensidade energética não pode ser vazia'
    assert len(dict_mandato_emissao_por_ano) > 0, 'O mandato de emissão por ano não pode ser vazio'
    
    anos = dict_mandato_emissao_por_ano.keys()
    df_iteracoes = pd.DataFrame()
    for nome_materia_prima, lista_fator_materia_prima in dict_rota_materia_prima.items():
        for fator_materia_prima in lista_fator_materia_prima:
            for elasticidade_iteracao in lista_elasticidades:
                for nome_demanda_intensidade_energetica in lista_nome_demanda_intensidade_energetica:
                    demanda_energetica_ano = [potencial_energetico[potencial_energetico['Ano'] == ano][nome_demanda_intensidade_energetica].values[0] for ano in anos]
                    
                    
                    numero_iteracao = 0
                    quanto_combustivel_eh_saf = np.zeros(len(anos))
                    while numero_iteracao < 20:
                        df_iteracoes = iteracao(df_iteracao=df_iteracoes,
                                                nome_demanda_intensidade_energetica=nome_demanda_intensidade_energetica,
                                                numero_iteracao=numero_iteracao,
                                                nome_preco_saf=nome_materia_prima,
                                                elasticidade_preco=elasticidade_iteracao,
                                                nome_fator_emissao_saf=fator_materia_prima,
                                                demanda_energetica_anterior = demanda_energetica_ano,
                                                quanto_combustivel_eh_saf=quanto_combustivel_eh_saf,
                                                mandato_emissao_por_ano = dict_mandato_emissao_por_ano,
                                                ano_referencia=ano_referencia)
                        
                        quanto_combustivel_eh_saf = df_iteracoes[df_iteracoes['numero_iteracao'] == numero_iteracao]['percentual_energetico_saf'].copy().reset_index(drop=True)
                        demanda_energetica_ano    = df_iteracoes[df_iteracoes['numero_iteracao'] == numero_iteracao]['demanda_intensidade_energetica'].copy().reset_index(drop=True)
                        numero_iteracao += 1
    return df_iteracoes.sort_values(by=['nome_demanda_intensidade_energetica', 'nome_preco_saf', 'fator_emissao_saf', 'elasticidade_preco', 'Ano', 'numero_iteracao'], axis=0).reset_index(drop=True)

In [47]:
lista_nome_demanda_intensidade_energetica = todas_demandas_intensidade_energetica
#lista_nome_demanda_intensidade_energetica = ['BAU-Base']
iteracoes = iteracao_dissertacao(lista_nome_demanda_intensidade_energetica=lista_nome_demanda_intensidade_energetica, 
                                 dict_rota_materia_prima=dict_rota_materia_prima,
                                #  dict_preco_saf_real_litro=dict_preco_saf_real_litro,
                                #  dict_fator_emissao_safs=dict_fator_emissao_safs,
                                 lista_elasticidades=dict_elasticidade_preco.values(), 
                                 dict_mandato_emissao_por_ano=dict_mandato_emissao_por_ano)
iteracoes

,elasticidade_preco,nome_preco_saf,fator_emissao_saf,nome_demanda_intensidade_energetica,Ano,mandato_emissao,numero_iteracao,demanda_intensidade_energetica,emissao_tudo_qav,reducao_emissao_referencia,reducao_via_saf_ano_referencia,reducao_via_saf_ano_destino,demanda_energetica_saf,variacao_demanda,percentual_energetico_saf
0,-1.59,atj_cana,atj_cana_com_luc,BAU-Acelerada,2030,0.03,0,2.786133e+11,2.479658e+13,7.254451e+11,6.150821e+11,1.340527e+12,3.088772e+10,-0.000000,0.110862
1,-1.59,atj_cana,atj_cana_com_luc,BAU-Acelerada,2030,0.03,1,2.654815e+11,2.362785e+13,7.254451e+11,-5.536484e+11,1.717967e+11,3.958449e+09,-0.070694,0.014910
2,-1.59,atj_cana,atj_cana_com_luc,BAU-Acelerada,2030,0.03,2,2.649235e+11,2.357819e+13,7.254451e+11,-6.033151e+11,1.221300e+11,2.814056e+09,-0.051407,0.010622
3,-1.59,atj_cana,atj_cana_com_luc,BAU-Acelerada,2030,0.03,3,2.645267e+11,2.354288e+13,7.254451e+11,-6.386230e+11,8.682206e+10,2.000508e+09,-0.037150,0.007563
4,-1.59,atj_cana,atj_cana_com_luc,BAU-Acelerada,2030,0.03,4,2.642447e+11,2.351778e+13,7.254451e+11,-6.637234e+11,6.172168e+10,1.422158e+09,-0.026725,0.005382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17815,-1.29,hefa_soja,hefa_soja_sem_luc,Retração-Moderada,2050,0.23,15,2.101585e+11,1.870411e+13,5.561746e+12,-5.477393e+12,8.435272e+10,1.224278e+09,-0.001531,0.005825
17816,-1.29,hefa_soja,hefa_soja_sem_luc,Retração-Moderada,2050,0.23,16,2.098845e+11,1.867972e+13,5.561746e+12,-5.501779e+12,5.996623e+10,8.703372e+08,-0.001090,0.004147
17817,-1.29,hefa_soja,hefa_soja_sem_luc,Retração-Moderada,2050,0.23,17,2.096897e+11,1.866239e+13,5.561746e+12,-5.519116e+12,4.262991e+10,6.187215e+08,-0.000776,0.002951
17818,-1.29,hefa_soja,hefa_soja_sem_luc,Retração-Moderada,2050,0.23,18,2.095513e+11,1.865006e+13,5.561746e+12,-5.531440e+12,3.030554e+10,4.398483e+08,-0.000552,0.002099


In [48]:
iteracoes[(iteracoes['Ano'] == 2030) & (iteracoes['nome_demanda_intensidade_energetica'] == 'BAU-Base') & (iteracoes['nome_preco_saf'] == 'valor_hefa_soja')]

,elasticidade_preco,nome_preco_saf,fator_emissao_saf,nome_demanda_intensidade_energetica,Ano,mandato_emissao,numero_iteracao,demanda_intensidade_energetica,emissao_tudo_qav,reducao_emissao_referencia,reducao_via_saf_ano_referencia,reducao_via_saf_ano_destino,demanda_energetica_saf,variacao_demanda,percentual_energetico_saf


In [49]:
iteracoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17820 entries, 0 to 17819
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   elasticidade_preco                   17820 non-null  float64
 1   nome_preco_saf                       17820 non-null  object 
 2   fator_emissao_saf                    17820 non-null  object 
 3   nome_demanda_intensidade_energetica  17820 non-null  object 
 4   Ano                                  17820 non-null  int64  
 5   mandato_emissao                      17820 non-null  float64
 6   numero_iteracao                      17820 non-null  int64  
 7   demanda_intensidade_energetica       17820 non-null  float64
 8   emissao_tudo_qav                     17820 non-null  float64
 9   reducao_emissao_referencia           17820 non-null  float64
 10  reducao_via_saf_ano_referencia       17820 non-null  float64
 11  reducao_via_saf_ano_destino 

In [50]:
iteracoes[iteracoes['numero_iteracao'] == 1]

,elasticidade_preco,nome_preco_saf,fator_emissao_saf,nome_demanda_intensidade_energetica,Ano,mandato_emissao,numero_iteracao,demanda_intensidade_energetica,emissao_tudo_qav,reducao_emissao_referencia,reducao_via_saf_ano_referencia,reducao_via_saf_ano_destino,demanda_energetica_saf,variacao_demanda,percentual_energetico_saf
1,-1.59,atj_cana,atj_cana_com_luc,BAU-Acelerada,2030,0.03,1,2.654815e+11,2.362785e+13,7.254451e+11,-5.536484e+11,1.717967e+11,3.958449e+09,-0.070694,0.014910
21,-1.59,atj_cana,atj_cana_com_luc,BAU-Acelerada,2040,0.13,1,2.437381e+11,2.169269e+13,3.143595e+12,-2.488815e+12,6.547799e+11,1.508709e+10,-0.240007,0.061899
41,-1.59,atj_cana,atj_cana_com_luc,BAU-Acelerada,2050,0.23,1,2.223694e+11,1.979088e+13,5.561746e+12,-4.390623e+12,1.171122e+12,2.698438e+10,-0.382378,0.121349
61,-1.47,atj_cana,atj_cana_com_luc,BAU-Acelerada,2030,0.03,1,2.670057e+11,2.376351e+13,7.254451e+11,-4.179947e+11,3.074504e+11,7.084110e+09,-0.065359,0.026532
81,-1.47,atj_cana,atj_cana_com_luc,BAU-Acelerada,2040,0.13,1,2.495473e+11,2.220971e+13,3.143595e+12,-1.971790e+12,1.171806e+12,2.700013e+10,-0.221893,0.108196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17721,-1.47,hefa_soja,hefa_soja_sem_luc,Retração-Moderada,2040,0.13,1,2.963311e+11,2.637347e+13,3.143595e+12,2.191963e+12,5.335559e+12,7.743917e+10,-0.076018,0.261327
17741,-1.47,hefa_soja,hefa_soja_sem_luc,Retração-Moderada,2050,0.23,1,3.164359e+11,2.816280e+13,5.561746e+12,3.981295e+12,9.543040e+12,1.385057e+11,-0.121112,0.437705
17761,-1.29,hefa_soja,hefa_soja_sem_luc,Retração-Moderada,2030,0.03,1,2.800638e+11,2.492567e+13,7.254451e+11,7.441726e+11,1.469618e+12,2.132972e+10,-0.019649,0.076160
17781,-1.29,hefa_soja,hefa_soja_sem_luc,Retração-Moderada,2040,0.13,1,2.993164e+11,2.663916e+13,3.143595e+12,2.457655e+12,5.601250e+12,8.129536e+10,-0.066710,0.271603


In [52]:
iteracoes.to_excel('iteracoes.xlsx', index=False)